In [3]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import torch
from matplotlib import pyplot as plt
import numpy as np
%matplotlib

Using matplotlib backend: <object object at 0x000001F5F501CA90>


In [5]:
base_dir = Path().absolute().parent
data_yaml_path = base_dir / "data/raw/corner-detection/data.yaml"
corner_detector_pt = base_dir / "notebooks/runs/detect/yolov8n_corner_detector/weights/best.pt"


In [4]:
# Enable GPU for training
# if torch.cuda.is_available():
#     torch.cuda.set_device(0)
#     print("CUDA on GPU")
# else:
#     print("CUDA on CPU")


In [14]:
yolo_model = YOLO('yolov8n.pt')

In [15]:
results = yolo_model.train(data=str(data_yaml_path),imgsz=416,epochs=100,batch=16,name='yolov8n_corner_detector')

Ultralytics YOLOv8.0.200  Python-3.8.18 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\ANOOP\Desktop\Python\ai-porfolio\otb-chess-vision\data\raw\corner-detection\data.yaml, epochs=100, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_corner_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, 

In [16]:
results.save_dir

WindowsPath('runs/detect/yolov8n_corner_detector')

In [28]:
results.save_dir

AttributeError: 'list' object has no attribute 'save_dir'

In [6]:
model_trained = YOLO(str(corner_detector_pt))

In [7]:
test_data_path = base_dir / "data/raw/corner-detection/test/images"

In [8]:
test_images=[]
for img_path in test_data_path.glob("*"):
    img = cv2.imread(img_path)
    test_images.append(img)

In [10]:
len(test_images)

5

In [11]:
results = model_trained.predict(source=test_images, line_thickness=1, conf=0.40, save_txt=True, save=True)

WARNING  'line_thickness' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'line_width' instead.

0: 416x416 4 corners, 1: 416x416 4 corners, 2: 416x416 3 corners, 3: 416x416 1 corner, 4: 416x416 4 corners, 496.3ms
Speed: 2.0ms preprocess, 99.3ms inference, 2.2ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs\detect\predict3
5 labels saved to runs\detect\predict3\labels


In [49]:
result_obj = model_trained(test_images[1])


0: 416x416 4 corners, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)


In [50]:
result_obj[0].boxes.xyxy

tensor([[358.9108, 361.5532, 379.6696, 381.3997],
        [ 12.9542, 359.6315,  29.2475, 376.3036],
        [ 83.8404,  23.9023,  98.6456,  39.0987],
        [337.1014,  20.1113, 351.1104,  34.7653]])

In [14]:
for result in result_obj:
    xywh=result.boxes.xywh
    print(xywh)
    xy = xywh[:,0:2]
    print(xy)
    img= result.orig_img
    for point in xy:
        cv2.circle(img, (int(point[0]),int(point[1])), radius=2, color=(255, 0, 0), thickness=-1)
cv2.imwrite("points_on_img2.jpg",img)


tensor([[ 26.4250, 376.7617,  22.7070,  22.1157],
        [ 63.8663,  24.7283,  14.6889,  14.7735],
        [380.6992, 377.8528,  17.3972,  18.3358],
        [321.0237,  35.9504,  16.2577,  16.0412]])
tensor([[ 26.4250, 376.7617],
        [ 63.8663,  24.7283],
        [380.6992, 377.8528],
        [321.0237,  35.9504]])


True

In [166]:
np.zeros((8,8,2))

array([[[          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0]],

       [[          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0]],

       [[          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0],
        [          0,           0]],

       [[          0,           0],
        [          0,           0],
        [          0,           0],
        [          0, 

In [11]:
def find_anchor_points(xys):
    board = np.zeros((9,9,2))
    temp = xys.numpy().copy()
    x_sorted = temp[:,0].argsort()
    
    # Get the Left and Right side anchor points
    left_anchors = temp[x_sorted[:2]]
    right_anchors = temp[x_sorted[2:]]
    
    # Determine the top and bottom corners
    top_left = left_anchors[left_anchors[:,1].argmax()]
    bottom_left = left_anchors[left_anchors[:,1].argmin()]
    
    top_right = right_anchors[right_anchors[:,1].argmax()]
    bottom_right = right_anchors[right_anchors[:,1].argmin()]
    
    print(top_left)
    print(top_right)
    print(bottom_left)
    print(bottom_right)
    # Fill the board with the corners based on thier locations
    board[0,0],board[8,0],board[0,8],board[8,8]=top_left,bottom_left,top_right,bottom_right
    return board
        
    

In [67]:
xy = result_obj[0].boxes.xywh[:,0:2]

In [36]:
def test_print_image(imgs,board,col,print=False,name="test_image.jpg"):
    imag = imgs.copy() #
    for pnt in board.reshape(81,2):
        cv2.circle(imag, (int(pnt[0]),int(pnt[1])), radius=5, color=col, thickness=-1)
    if print == True:
        cv2.imwrite(name,imag)
    else:
        return imag

In [68]:
board = find_anchor_points(xy)
snap_shot = board.copy() 
test_image = test_print_image(test_images[1].copy(),board,col=(255,0,0))

[     21.101      367.97]
[     369.29      371.48]
[     91.243        31.5]
[     344.11      27.438]


In [33]:
len(board[7])

9

In [59]:
test = len(board[7])-2
for i in range(1,test+1):
    print(i)
    
[(1,2) for i in range(1,test+1) ]

1
2
3
4
5
6


[(1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2)]

In [37]:
def eculidean_distance(p1,p2):
    temp = p2-p1
    dist = np.sqrt(np.dot(temp,temp.T))
    return dist

In [39]:
for i in range(1,9):
    print(i)

1
2
3
4
5
6
7
8


In [71]:
def interpolate_points(board_section):
    print(f"Board Section state:{board_section}")
    start_pnt = board_section[0]
    end_pnt = board_section[-1]
    interpolations_required = len(board_section) - 1
    print(end_pnt[0],start_pnt[0])
    # Unit Changes in X and Y direction for given line
    dx = (end_pnt[0] - start_pnt[0])/ (interpolations_required - 1)  
    dy = (end_pnt[1] - start_pnt[1])/ (interpolations_required - 1)
    
    print(dx,dy)
    
    # Interpolate missing points
    #return [( start_pnt[0] + (dx * i) , start_pnt[1] + (dy * i)) for i in range(1,9)]
    print(board_section[1:-1])
    board_section[1:-1]= [( start_pnt[0] + (dx * i) , start_pnt[1] + (dy * i)) for i in range(1,interpolations_required)]
    print(f"Interpolated Board Section :{board_section}")
    return board_section

In [61]:
interpolate_points(board[8])

Board Section state:[[     63.866      24.728]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     321.02       35.95]]
321.023681640625 63.86625289916992
28.573047637939453 1.2468986511230469
[[          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]]
Interpolated Board Section :[[     63.866      24.728]
 [     92.439      25.975]
 [     121.01      27.222]
 [     149.59      28.469]
 [     178.16      29.716]
 [     206.73      30.963]
 [      235.3       32.21]
 [     263.88      33.457]
 [     321.02       35.95]]


array([[     63.866,      24.728],
       [     92.439,      25.975],
       [     121.01,      27.222],
       [     149.59,      28.469],
       [     178.16,      29.716],
       [     206.73,      30.963],
       [      235.3,       32.21],
       [     263.88,      33.457],
       [     321.02,       35.95]])

In [52]:
board[8]

array([[     63.866,      24.728],
       [     92.439,      25.975],
       [     121.01,      27.222],
       [     149.59,      28.469],
       [     178.16,      29.716],
       [     206.73,      30.963],
       [      235.3,       32.21],
       [     263.88,      33.457],
       [     292.45,      34.703]])

In [190]:
interpolate_points(board[8])

Board Section state:[[     63.866      24.728]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     321.02       35.95]]
321.023681640625 63.86625289916992
28.573047637939453 1.2468986511230469
[[          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]]
Interpolated Board Section :[[     63.866      24.728]
 [     92.439      25.975]
 [     121.01      27.222]
 [     149.59      28.469]
 [     178.16      29.716]
 [     206.73      30.963]
 [      235.3       32.21]
 [     263.88      33.457]
 [     321.02       35.95]]


array([[     63.866,      24.728],
       [     92.439,      25.975],
       [     121.01,      27.222],
       [     149.59,      28.469],
       [     178.16,      29.716],
       [     206.73,      30.963],
       [      235.3,       32.21],
       [     263.88,      33.457],
       [     321.02,       35.95]])

In [180]:
interpolate_points(board[7])

Board Section state:[[     55.546      102.96]
 [     86.517      103.95]
 [     117.49      104.95]
 [     148.46      105.95]
 [     179.43      106.94]
 [      210.4      107.94]
 [     241.37      108.94]
 [     272.34      109.94]
 [     334.28      111.93]]
334.284918891059 55.54598066541884
30.970993136182244 0.9967506785451659
[[     86.517      103.95]
 [     117.49      104.95]
 [     148.46      105.95]
 [     179.43      106.94]
 [      210.4      107.94]
 [     241.37      108.94]
 [     272.34      109.94]]
Interpolated Board Section :[[     55.546      102.96]
 [     86.517      103.95]
 [     117.49      104.95]
 [     148.46      105.95]
 [     179.43      106.94]
 [      210.4      107.94]
 [     241.37      108.94]
 [     272.34      109.94]
 [     334.28      111.93]]


array([[     55.546,      102.96],
       [     86.517,      103.95],
       [     117.49,      104.95],
       [     148.46,      105.95],
       [     179.43,      106.94],
       [      210.4,      107.94],
       [     241.37,      108.94],
       [     272.34,      109.94],
       [     334.28,      111.93]])

In [53]:
board[0]

array([[     26.425,      376.76],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [      380.7,      377.85]])

In [54]:
interpolate_points(board[0])

Board Section state:[[     26.425      376.76]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [      380.7      377.85]]
380.6992492675781 26.42502784729004
39.36380238003201 0.12123277452256945
[[          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]]
Interpolated Board Section :[[     26.425      376.76]
 [     65.789      376.88]
 [     105.15         377]
 [     144.52      377.13]
 [     183.88      377.25]
 [     223.24      377.37]
 [     262.61      377.49]
 [     301.97      377.61]
 [     341.34      377.73]]


array([[     26.425,      376.76],
       [     65.789,      376.88],
       [     105.15,         377],
       [     144.52,      377.13],
       [     183.88,      377.25],
       [     223.24,      377.37],
       [     262.61,      377.49],
       [     301.97,      377.61],
       [     341.34,      377.73]])

In [88]:
board[0]

array([[     26.425,      376.76],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [          0,           0],
       [      380.7,      377.85]])

In [15]:
def fill_board(board):
    print("Start of board fill")
    # Fill the left and right sections of the board first (Vertical Interpolation)
    print(f"Left section{board[:,0]}")
    print(f"Right section{board[:,8]}")
    left_section = interpolate_points(board[:,0])
    right_section = interpolate_points(board[:,8])
    board[:,0] = left_section
    board[:,8] = right_section
    print(f"After inter Left section{left_section}")
    print(f"After inter  Right section{right_section}")
    # Interpolate horizontally for each horizontal section
    for i,hor_section in enumerate(board):
        #print(f"horizontal section:{hor_section}")
        interpolated_section =  interpolate_points(hor_section) 
        board[i] = interpolated_section
    return board

In [69]:
board = find_anchor_points(xy)
snap_shot = board.copy() 
filled_board = fill_board(board)

[     21.101      367.97]
[     369.29      371.48]
[     91.243        31.5]
[     344.11      27.438]
Start of board fill
Left section[[     21.101      367.97]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     91.243        31.5]]
Right section[[     369.29      371.48]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     344.11      27.438]]
Board Section state:[[     21.101      367.97]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     91.243        31.5]]
91.24302673339844 21.10085105895996
4.383885979652405 -21.029194116592407
[[          0           0

In [65]:
filled_board[8]

array([[     63.866,      24.728],
       [     96.011,      26.131],
       [     128.16,      27.534],
       [      160.3,      28.937],
       [     192.44,      30.339],
       [     224.59,      31.742],
       [     256.73,      33.145],
       [     288.88,      34.548],
       [     321.02,       35.95]])

In [161]:
filled_board_C = filled_board.copy()

In [163]:
filled_board[1]

array([[     31.105,      332.76],
       [     73.872,      333.05],
       [     116.64,      333.35],
       [     159.41,      333.64],
       [     202.17,      333.94],
       [     244.94,      334.23],
       [     287.71,      334.53],
       [     373.24,      335.12]])

In [162]:
filled_board_C.reshape(64,2)

array([[     26.425,      376.76],
       [     70.709,       376.9],
       [     114.99,      377.03],
       [     159.28,      377.17],
       [     203.56,      377.31],
       [     247.85,      377.44],
       [     292.13,      377.58],
       [      380.7,      377.85],
       [     31.105,      332.76],
       [     73.872,      333.05],
       [     116.64,      333.35],
       [     159.41,      333.64],
       [     202.17,      333.94],
       [     244.94,      334.23],
       [     287.71,      334.53],
       [     373.24,      335.12],
       [     35.785,      288.75],
       [     77.035,      289.21],
       [     118.28,      289.66],
       [     159.53,      290.11],
       [     200.78,      290.57],
       [     242.03,      291.02],
       [     283.28,      291.47],
       [     365.78,      292.38],
       [     40.465,      244.75],
       [     80.197,      245.36],
       [     119.93,      245.97],
       [     159.66,      246.58],
       [     199.39,

In [174]:
for pnt in filled_board:
    print(pnt)

[[     26.425      376.76]
 [     65.789      376.88]
 [     105.15         377]
 [     144.52      377.13]
 [     183.88      377.25]
 [     223.24      377.37]
 [     262.61      377.49]
 [     301.97      377.61]
 [      380.7      377.85]]
[[     30.585      337.65]
 [      68.75      337.89]
 [     106.91      338.14]
 [     145.08      338.39]
 [     183.24      338.63]
 [     221.41      338.88]
 [     259.57      339.12]
 [     297.74      339.37]
 [     374.07      339.86]]
[[     34.745      298.53]
 [     71.711       298.9]
 [     108.68      299.27]
 [     145.64      299.65]
 [     182.61      300.02]
 [     219.57      300.39]
 [     256.54      300.76]
 [     293.51      301.13]
 [     367.44      301.87]]
[[     38.905      259.42]
 [     74.672      259.91]
 [     110.44      260.41]
 [     146.21      260.91]
 [     181.97       261.4]
 [     217.74       261.9]
 [     253.51       262.4]
 [     289.27      262.89]
 [     360.81      263.89]]
[[     43.066       220.

In [70]:
test_print_image(test_image,filled_board,col=(0,255,0),name="Test_img1_gird_x2_dist.jpg",print=True)

In [82]:
plt.imshow(test_images[4])

: 

In [ ]:
def build_board(corners):
    if corners is not None:
        if len(corners) >= 4:
            # Build the board with the points
            print("Corners detected")
            board_corners= non_max_supression(corners)
            
            # Detect the pieces
        else:
            # Detect Pieces Only